In [1]:
import pandas as pd
import requests
import datetime as dt
from dateutil import parser
import pytz
import json
import base64

In [2]:
OPENFX_URL = "https://marginalttdemowebapi.fxopen.net/api/v2"

API_ID = "fecc426d-1e89-4b52-9052-8c429a23a5b8"
API_KEY = "YRbJYmff8ybacWe3"
API_SECRET = "2FA4JybcNzKQhzMKb2jZFdXY3qNxH34RkDGRSXqEjGBJXQ9wnMJaGSkWSKasTPCX"

SECURE_HEADER = {
    "Authorization": f"Basic {API_ID}:{API_KEY}:{API_SECRET}",
    "Content-Type": "application/json",
    "Accept": "application/json",
}

In [3]:
session = requests.Session()
session.headers.update(SECURE_HEADER)

In [4]:
full_url = lambda x: f'{OPENFX_URL}/{x}'

## Account Details

In [5]:
resp = session.get(full_url('account'))

In [6]:
resp.status_code

200

In [7]:
print(json.dumps(resp.json(), indent=2))

{
  "Id": 28501722,
  "AccountingType": "Gross",
  "Name": "cipher one",
  "FirstName": "cipher",
  "LastName": "one",
  "Phone": "+917795293598",
  "Country": "INDIA",
  "State": "",
  "City": "",
  "Address": "",
  "ZipCode": "",
  "Email": "sohailkustagi1234@gmail.com",
  "Comment": "",
  "Registered": 1731698894471,
  "Modified": 1736771600816,
  "IsArchived": false,
  "IsBlocked": false,
  "IsReadonly": false,
  "IsValid": true,
  "IsWebApiEnabled": true,
  "Leverage": 33,
  "Balance": 5000.0,
  "BalanceCurrency": "USD",
  "Profit": 0.0,
  "Commission": 0.0,
  "AgentCommission": 0.0,
  "Swap": 0.0,
  "Rebate": 0.0,
  "Equity": 5000.0,
  "Margin": 0.0,
  "MarginCallLevel": 100,
  "StopOutLevel": 50,
  "ReportCurrency": "USD",
  "IsTokenCommissionEnabled": false,
  "Throttling": [
    {
      "Protocol": "Rest",
      "SessionsPerAccount": 2,
      "RequestsPerSecond": 5,
      "ThrottlingMethods": [],
      "ConcurrentRequestCount": 100
    }
  ],
  "IsLongOnly": false
}


## Symbols (Instruments)

In [8]:
# first get all of the symbols
resp = session.get(full_url('symbol'))
symbol_data = resp.json()

# print first 2, note the StatusGroupId=="Forex"
[print(json.dumps(x, indent=2)) for x in symbol_data[:2]]

# also we only want symbols where we can also load history data. For that there is the quotehistory/symbols
resp = session.get(full_url('quotehistory/symbols'))
his_symbol_data = resp.json()

print(his_symbol_data[:5])

# you can probaby see, some of the instruments are appended with "_L"
# in the API code we will filter for symbols that are in the symbol_data and are in his_symbol_data and do not have this L and have StatusGroupId=="Forex"

{
  "Symbol": "EURUSD",
  "Precision": 5,
  "IsTradeAllowed": true,
  "MarginMode": "Forex",
  "ProfitMode": "Forex",
  "ContractSize": 100000.0,
  "MarginHedged": 0.5,
  "MarginFactor": 1.0,
  "MarginCurrency": "EUR",
  "MarginCurrencyPrecision": 2,
  "ProfitCurrency": "USD",
  "ProfitCurrencyPrecision": 2,
  "Description": "Euro vs US Dollar",
  "Schedule": "Forex",
  "Color": -10496,
  "SwapEnabled": true,
  "SwapType": "Points",
  "SwapSizeShort": 0.24,
  "SwapSizeLong": -9.48,
  "TripleSwapDay": 3,
  "MinTradeAmount": 1000.0,
  "MaxTradeAmount": 100000000.0,
  "TradeAmountStep": 1000.0,
  "CommissionType": "Percentage",
  "CommissionChargeType": "PerLot",
  "Commission": 0.0018,
  "LimitsCommission": 0.0015,
  "MinCommission": 0.0,
  "MinCommissionCurrency": "USD",
  "DefaultSlippage": 0.01,
  "StatusGroupId": "Forex",
  "SecurityName": "ECN FX Group1",
  "SecurityDescription": "Major Forex symbols",
  "StopOrderMarginReduction": 1.0,
  "HiddenLimitOrderMarginReduction": 1.0,
  "I

## Perdiodicities (Granularities)

In [9]:
# for a given instrument (symbol) we can get the available candle granularities
resp = session.get(full_url('quotehistory/EURUSD/periodicities'))
print(resp.json())

['D1', 'H1', 'H4', 'M1', 'M15', 'M30', 'M5', 'MN1', 'S1', 'S10', 'W1']


## Candles

Candles are a little bit different than Oanda. <br><br>
The Good:
- Prices are floats!!
- Last available candle is in the response<br><br>

The bad:
- We have to make separate calls for ask and bid prices
- We have to specify a from date no matter what. It has to be a timestamp in ms format without timezone.
    - If the count we specify is negative it counts back from the date
    - If the count we specify is postive it counts forward from the date
- There is a 1000 candle limit per request

In [12]:
test_date = dt.datetime.utcnow()
past_date = parser.parse("2023-03-02T03:11:00")
print("test_date", test_date)
print("past_date", past_date)

test_date_ts = pd.Timestamp(test_date).timestamp()
past_date_ts = pd.Timestamp(past_date).timestamp()
print("test_date_ts", test_date_ts)
print("int(test_date_ts*1000)", int(test_date_ts*1000))
print("int(past_date_ts*1000)", int(past_date_ts*1000))

test_date 2025-01-13 13:44:44.934231
past_date 2023-03-02 03:11:00
test_date_ts 1736775884.934231
int(test_date_ts*1000) 1736775884934
int(past_date_ts*1000) 1677726660000


/tmp/ipykernel_32928/3870405082.py:1: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  test_date = dt.datetime.utcnow()


In [15]:
ts_conv = 1677726660000
pd.to_datetime(ts_conv, unit='ms')

Timestamp('2023-03-02 03:11:00')

In [16]:
LABEL_MAP = {
    'Open': 'o',
    'High': 'h',
    'Low': 'l',
    'Close': 'c',
}

# normal params
count = -10
granularity = "M15"
pair = "EURUSD"

# how far do we need to go back to get our candles
params = dict(
    timestamp=int(pd.Timestamp(dt.datetime.utcnow()).timestamp() * 1000),
    count=count
)

url = full_url(f'quotehistory/{pair}/{granularity}/bars/bid')
bid_data = session.get(url, params=params).json()

url = full_url(f'quotehistory/{pair}/{granularity}/bars/ask')
ask_data = session.get(url, params=params).json()


/tmp/ipykernel_32928/4013030328.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp=int(pd.Timestamp(dt.datetime.utcnow()).timestamp() * 1000),


In [17]:
# bid_data and ask_data will contain (hopefully) a key called "Bars", with the candle data:
bid_data["Bars"][:2] # the first two

[{'Timestamp': 1736767800000,
  'Open': 1.01908,
  'High': 1.02029,
  'Low': 1.01885,
  'Close': 1.01993,
  'Volume': 1625.0},
 {'Timestamp': 1736768700000,
  'Open': 1.01994,
  'High': 1.02,
  'Low': 1.01955,
  'Close': 1.01959,
  'Volume': 1077.0}]

In [18]:
# Now to convert them to a dataframe
# main points: Timestamp to a datetime.
# we'll need to make a DataFrame for ask, for bid, merge and calculate the mid

In [19]:
# a little utility to take in a candle and return it as and object
# for example, if we are working with bid prices
# price_label='bid'
# item= {'Volume': 1476, 'Close': 1.06064,  'Low': 1.06054,  'High': 1.06104,  'Open': 1.06081,  'Timestamp': 1677535200000}
# the returned object is: { 'time': datetime, 'bid_c': 1.06064,  'bid_l': 1.06054,  'bid_h': 1.06104,  'bid_o': 1.06081 }
def get_price_dict(price_label: str, item):
        data = dict(time=pd.to_datetime(item['Timestamp'], unit='ms'))
        for ohlc in LABEL_MAP.keys():
            data[f"{price_label}_{LABEL_MAP[ohlc]}"]=item[ohlc]
        return data

In [20]:
# let's make the lists of objects
AvailableTo = pd.to_datetime(bid_data['AvailableTo'], unit='ms')

bids = [get_price_dict('bid', item) for item in bid_data["Bars"]]
asks = [get_price_dict('ask', item) for item in ask_data["Bars"]]

In [21]:
# last 2
bids[-2:]

[{'time': Timestamp('2025-01-13 13:30:00'),
  'bid_o': 1.02087,
  'bid_h': 1.02209,
  'bid_l': 1.02087,
  'bid_c': 1.02118},
 {'time': Timestamp('2025-01-13 13:45:00'),
  'bid_o': 1.02122,
  'bid_h': 1.02161,
  'bid_l': 1.02113,
  'bid_c': 1.02145}]

In [22]:
# now merge on time - the assumption here is we have the same time values for both. it would be weird if we didn't
df_bid = pd.DataFrame.from_dict(bids)
df_ask = pd.DataFrame.from_dict(asks)
df_merged = pd.merge(left=df_bid, right=df_ask, on='time')    

In [23]:
df_merged

,time,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2025-01-13 11:30:00,1.01908,1.02029,1.01885,1.01993,1.01910,1.02030,1.01887,1.01995
1,2025-01-13 11:45:00,1.01994,1.02000,1.01955,1.01959,1.01995,1.02002,1.01956,1.01960
2,2025-01-13 12:00:00,1.01960,1.01980,1.01903,1.01947,1.01962,1.01982,1.01904,1.01948
3,2025-01-13 12:15:00,1.01946,1.01950,1.01878,1.01888,1.01948,1.01951,1.01879,1.01888
4,2025-01-13 12:30:00,1.01887,1.01964,1.01886,1.01943,1.01889,1.01965,1.01888,1.01944
5,2025-01-13 12:45:00,1.01942,1.02061,1.01940,1.02028,1.01943,1.02063,1.01940,1.02029
6,2025-01-13 13:00:00,1.02029,1.02033,1.01964,1.02004,1.02030,1.02035,1.01965,1.02005
7,2025-01-13 13:15:00,1.02006,1.02131,1.01993,1.02086,1.02006,1.02132,1.01993,1.02086
8,2025-01-13 13:30:00,1.02087,1.02209,1.02087,1.02118,1.02087,1.02210,1.02087,1.02119
9,2025-01-13 13:45:00,1.02122,1.02161,1.02113,1.02145,1.02123,1.02162,1.02114,1.02148


In [24]:
# FINALLY calcuate the mid, and we are done
for i in ['_o', '_h', '_l', '_c']:
    df_merged[f'mid{i}'] = (df_merged[f'ask{i}'] - df_merged[f'bid{i}']) / 2 + df_merged[f'bid{i}']

In [25]:
df_merged

,time,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,mid_o,mid_h,mid_l,mid_c
0,2025-01-13 11:30:00,1.01908,1.02029,1.01885,1.01993,1.01910,1.02030,1.01887,1.01995,1.019090,1.020295,1.018860,1.019940
1,2025-01-13 11:45:00,1.01994,1.02000,1.01955,1.01959,1.01995,1.02002,1.01956,1.01960,1.019945,1.020010,1.019555,1.019595
2,2025-01-13 12:00:00,1.01960,1.01980,1.01903,1.01947,1.01962,1.01982,1.01904,1.01948,1.019610,1.019810,1.019035,1.019475
3,2025-01-13 12:15:00,1.01946,1.01950,1.01878,1.01888,1.01948,1.01951,1.01879,1.01888,1.019470,1.019505,1.018785,1.018880
4,2025-01-13 12:30:00,1.01887,1.01964,1.01886,1.01943,1.01889,1.01965,1.01888,1.01944,1.018880,1.019645,1.018870,1.019435
5,2025-01-13 12:45:00,1.01942,1.02061,1.01940,1.02028,1.01943,1.02063,1.01940,1.02029,1.019425,1.020620,1.019400,1.020285
6,2025-01-13 13:00:00,1.02029,1.02033,1.01964,1.02004,1.02030,1.02035,1.01965,1.02005,1.020295,1.020340,1.019645,1.020045
7,2025-01-13 13:15:00,1.02006,1.02131,1.01993,1.02086,1.02006,1.02132,1.01993,1.02086,1.020060,1.021315,1.019930,1.020860
8,2025-01-13 13:30:00,1.02087,1.02209,1.02087,1.02118,1.02087,1.02210,1.02087,1.02119,1.020870,1.022095,1.020870,1.021185
9,2025-01-13 13:45:00,1.02122,1.02161,1.02113,1.02145,1.02123,1.02162,1.02114,1.02148,1.021225,1.021615,1.021135,1.021465


In [26]:
if count < 0 and df_merged.shape[0] > 0 and df_merged.iloc[-1].time == AvailableTo:
    df_merged = df_merged[:-1]

In [27]:
df_merged

,time,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,mid_o,mid_h,mid_l,mid_c
0,2025-01-13 11:30:00,1.01908,1.02029,1.01885,1.01993,1.01910,1.02030,1.01887,1.01995,1.019090,1.020295,1.018860,1.019940
1,2025-01-13 11:45:00,1.01994,1.02000,1.01955,1.01959,1.01995,1.02002,1.01956,1.01960,1.019945,1.020010,1.019555,1.019595
2,2025-01-13 12:00:00,1.01960,1.01980,1.01903,1.01947,1.01962,1.01982,1.01904,1.01948,1.019610,1.019810,1.019035,1.019475
3,2025-01-13 12:15:00,1.01946,1.01950,1.01878,1.01888,1.01948,1.01951,1.01879,1.01888,1.019470,1.019505,1.018785,1.018880
4,2025-01-13 12:30:00,1.01887,1.01964,1.01886,1.01943,1.01889,1.01965,1.01888,1.01944,1.018880,1.019645,1.018870,1.019435
5,2025-01-13 12:45:00,1.01942,1.02061,1.01940,1.02028,1.01943,1.02063,1.01940,1.02029,1.019425,1.020620,1.019400,1.020285
6,2025-01-13 13:00:00,1.02029,1.02033,1.01964,1.02004,1.02030,1.02035,1.01965,1.02005,1.020295,1.020340,1.019645,1.020045
7,2025-01-13 13:15:00,1.02006,1.02131,1.01993,1.02086,1.02006,1.02132,1.01993,1.02086,1.020060,1.021315,1.019930,1.020860
8,2025-01-13 13:30:00,1.02087,1.02209,1.02087,1.02118,1.02087,1.02210,1.02087,1.02119,1.020870,1.022095,1.020870,1.021185


In [ ]:
# and breathe...

## Latest Prices

In [28]:
# here the endpoint needs space separated instruments rather than comma. Yes, in a URL that is a bit weird.
instruments_list = ["GBPJPY", "EURUSD", "EURNOK"]
url = full_url(f"tick/{' '.join(instruments_list)}")
print(url)

https://marginalttdemowebapi.fxopen.net/api/v2/tick/GBPJPY EURUSD EURNOK


In [29]:
prices = session.get(url)
price_data = prices.json()

# you can see below that there are some differences to the Oanda Api in what comes back, imho this is a lot better
price_data

[{'Symbol': 'EURUSD',
  'Timestamp': 1736776383998,
  'BestBid': {'Type': 'Bid', 'Price': 1.02118, 'Volume': 50000000.0},
  'BestAsk': {'Type': 'Ask', 'Price': 1.02119, 'Volume': 50000000.0},
  'IndicativeTick': False,
  'TickType': 'Normal'},
 {'Symbol': 'GBPJPY',
  'Timestamp': 1736776384099,
  'BestBid': {'Type': 'Bid', 'Price': 190.796, 'Volume': 50000000.0},
  'BestAsk': {'Type': 'Ask', 'Price': 190.809, 'Volume': 50000000.0},
  'IndicativeTick': False,
  'TickType': 'Normal'},
 {'Symbol': 'EURNOK',
  'Timestamp': 1736776372775,
  'BestBid': {'Type': 'Bid', 'Price': 11.71217, 'Volume': 50000000.0},
  'BestAsk': {'Type': 'Ask', 'Price': 11.71306, 'Volume': 50000000.0},
  'IndicativeTick': False,
  'TickType': 'Normal'}]